In [1]:
import json
from glob import glob
from os.path import join as pjoin
import os
import cv2
import numpy as np

In [2]:
from match_elements.GUI_pair import GUIPair
from GUI import GUI
from Operation import Operation

In [3]:
from keras.applications.resnet import ResNet50
resnet_model = ResNet50(include_top=False, input_shape=(32, 32, 3))

In [4]:
from paddleocr import PaddleOCR
paddle_ocr = PaddleOCR(use_angle_cls=True, lang='en')

[2022/04/11 12:37:58] root WARNING: lang en is not support in PP-OCRv2, auto switch to version PP-OCR
[2022/04/11 12:37:58] root WARNING: lang en is not support in PP-OCRv2, auto switch to version PP-OCR
[2022/04/11 12:37:58] root WARNING: version PP-OCRv2 not support cls models, auto switch to version PP-OCR
Namespace(benchmark=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/home/ml/.paddleocr/2.3.0.2/ocr/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='/home/ml/.paddleocr/2.3.0.2/ocr/det/en/en_ppocr_mobile_v2.0_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='box', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_s

In [5]:
from ppadb.client import Client as AdbClient
client = AdbClient(host="127.0.0.1", port=5037)

In [6]:
def cap_device_current_gui(device, paddle_ocr, screen_path="data/output/screen.jpg", is_detect=True):
    screen = device.screencap()
    with open(screen_path, "wb") as fp:
        fp.write(screen)
        
    gui = GUI(screen_path)
    if is_detect:
        gui.detect_element(True, True, True, paddle_cor=paddle_ocr)
    else:
        gui.load_detection_result()
    return gui

In [21]:
def match_element_in_gui(target_element, gui, device, resnet_model, scroll_down=True):
    matched_ele = gui.match_elements(target_element.clip, resnet_model, target_element.text_content, min_similarity_img=0.55, show=True)
    # scroll down if current gui is not matched
    if matched_ele is None and scroll_down:
        print('Scroll down and try to match again')
        # device.shell('input roll' + str(device.wm_size().height - 50) + ' 0')
        device.input_swipe(50, (device.wm_size().height * 0.9), 50, 20, 500)
        next_gui = cap_device_current_gui(device, paddle_ocr)
        # cv2.imshow('img', next_gui.img)
        # next_gui.show_detection_result()
        match_element_in_gui(target_element, next_gui, device, resnet_model, scroll_down=False)
    return matched_ele

In [16]:
dev1 = client.device('emulator-5554')
gui1 = cap_device_current_gui(dev1, paddle_ocr, screen_path='data/screen/80.png', is_detect=True)
gui1.show_detection_result()
# gui1 = GUI('data/screen/80.png')
# gui1.detect_element(True, True, True, paddle_cor=paddle_ocr)
# gui1.load_detection_result()

In [17]:
dev2 = client.device('emulator-5558')
gui2 = cap_device_current_gui(dev2, paddle_ocr, screen_path='data/screen/60.png', is_detect=True)
gui2.show_detection_result()

In [18]:
dev3 = client.device('emulator-5560')
gui3 = cap_device_current_gui(dev3, paddle_ocr, screen_path='data/screen/50.png', is_detect=True)
gui3.show_detection_result()

In [19]:
dev4 = client.device('emulator-5556')
gui4 = cap_device_current_gui(dev4, paddle_ocr, screen_path='data/screen/37.png', is_detect=True)
gui4.show_detection_result()

In [12]:
e = gui2.get_element_by_clicking()

Click (287, 354)


In [13]:
matched_e1 = match_element_in_gui(e, gui1, dev1, resnet_model)
# matched_e2 = match_element_in_gui(e, gui2, dev2, resnet_model)
matched_e3 = match_element_in_gui(e, gui3, dev3, resnet_model)
matched_e4 = match_element_in_gui(e, gui4, dev4, resnet_model)

Match by image
Match by image
No matched element found
Scroll down and try to match again
[2022/04/11 12:20:12] root DEBUG: dt_boxes num : 18, elapse : 0.1713252067565918
[2022/04/11 12:20:12] root DEBUG: cls num  : 18, elapse : 0.06683468818664551
[2022/04/11 12:20:12] root DEBUG: rec_res num  : 18, elapse : 0.11641359329223633
[Text Detection Completed in 0.371 s] Input: data/output/screen.jpg Output: data/output/ocr/screen.json
[Compo Detection Completed in 0.388 s] Input: data/output/screen.jpg Output: data/output/ip/screen.json
[Merge Completed] Input: data/output/screen.jpg Output: data/output/merge/screen.jpg
No matched element found


In [21]:
dev1.wm_size()

Size(width=2200, height=2480)

In [22]:
dev2.wm_size()

Size(width=1080, height=2280)

In [23]:
dev3.wm_size()

Size(width=1080, height=1920)

In [24]:
dev4.wm_size()

Size(width=480, height=800)